In [69]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Bidirectional, TimeDistributed,Conv1D, MaxPooling1D, Input, concatenate
from keras.layers.recurrent import SimpleRNN
from keras.layers import GRU, LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.models import Model
from keras.callbacks import EarlyStopping
import keras
import os
import tarfile
import re
import pandas as pd
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
import numpy as np

In [184]:
def RNN(maxlen = 50, max_features = 4590, embed_size =32):
    model = Sequential()
    model.add(Embedding(max_features, embed_size, input_length=maxlen))
#     model.add(Dropout(0.5))
    model.add(GRU(16,dropout=0.2))
#     model.add(Dense(8, activation='relu'))
#     model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    print(model.summary())

    return model

def CNN(maxlen=50, max_features=4590, embed_size=32):
    # Inputs
    comment_seq = Input(shape=[maxlen], name='x_seq')

    # Embeddings layers
    emb_comment = Embedding(max_features, embed_size)(comment_seq)

    # conv layers
    convs = []
#     filter_sizes = [2, 3, 4, 5]
    filter_sizes = [2, 3]

    for fsz in filter_sizes:
        l_conv = Conv1D(filters=10, kernel_size=fsz, activation='relu')(emb_comment)
        l_pool = MaxPooling1D(maxlen - fsz + 1)(l_conv)
        l_pool = Flatten()(l_pool)
        convs.append(l_pool)
    merge = concatenate(convs, axis=1)

    out = Dropout(0.2)(merge)
    output = Dense(16, activation='relu')(out)

    output = Dense(units=1, activation='sigmoid')(output)

    model = Model([comment_seq], output)
    print(model.summary())
    return model


def embedding(maxlen=50, max_features=4590, embed_size=16):
    # define the model
    model = Sequential()
    model.add(Embedding(max_features, embed_size, input_length=maxlen))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    # summarize the model
    print(model.summary())
    return model

    
def model_fit(model,padded_docs,labels,train_index,test_index, epochs = 5):
    len_train = len(train_index)
    len_test = len(test_index)
    x_test =  np.zeros(shape = (len_test,len(padded_docs[0])))
    y_test = np.zeros(shape = (len_test))
    x_train = np.zeros(shape = (len_train,len(padded_docs[0])))
    y_train = np.zeros(shape = (len_train))
    i = 0
    for tri in train_index:
        x_train[i] = padded_docs[tri]
        y_train[i] = labels[tri]
        i+=1
    j = 0
    for ti in test_index:
        x_test[j] = padded_docs[ti]
        y_test[j] =  labels[ti]
        j +=1
        
    print("Test set from ",test_index[0], " to ",test_index[-1])

    model.fit(x_train, y_train,epochs=epochs)
    eva = model.evaluate(x_test,y_test)
    print('loss: ',eva[0])
    print('accuracy: ',eva[1])
    print('precision: ',eva[2])
    print('recall: ',eva[3])
    print('f1-score: ',(2*eva[3]*eva[2])/(eva[3]+eva[2]))
    return model 

# def model_fit(model,x_train,y_train,x_test,y_test):
    
#     model.fit(x_train, y_train,epochs=50,verbose=0)
#     eva = model.evaluate(x_test,y_test)
#     print('loss: ',eva[0])
#     print('accuracy: ',eva[1])
#     print('precision: ',eva[2])
#     print('recall: ',eva[3])
#     print('f1-score: ',(2*eva[3]*eva[2])/(eva[3]+eva[2]))
#     return model 

In [185]:
# define documents
## read Data
SPL = "<SPL>"
path = "csv/trimedValues2.csv"
train_file = path
dataframe = pd.read_csv(path, na_filter = False)
docs = []
labels = []
for i,data in dataframe.iterrows():
    d = data["comment"]
    c = data["code"]
    s = []
    s =  d+" "+SPL+" "+c # comment and code together
    docs.append(s)
    l = 1 if data["non-information"] == "yes" else 0
    labels.append(l)
# integer encode the documents
vocab_size = 2000
encoded_docs = [one_hot(d, vocab_size) for d in docs]
# pad documents to a max length of 41 words
# The langest sentence contains 200 words but the Average is 21. If  padding all sentence to length of langest one, most of sentence
# will be 0. That is not good for RNN, so we set padding length to 50
summ = 0
for i in docs:
    summ += len(i.split())
print(summ/len(docs))
# max_length = len(max(docs, key=len).split())
max_length = 50
##
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

21.7673531655225


In [186]:
from sklearn.model_selection import KFold

train_index = range(1050)
test_index = range(1050,len(padded_docs))
model_CNN = CNN(max_length,vocab_size,32)
model_CNN.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',keras.metrics.Precision(),keras.metrics.Recall()])
model_CNN = model_fit(model_CNN,padded_docs,labels,train_index,test_index)

# n_split=5
# for train_index,test_index in KFold(n_split).split(padded_docs):
#     # define the model
#     model = CNN(max_length,vocab_size,32)
    
#     model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',keras.metrics.Precision(),keras.metrics.Recall()])

#     model = model_fit(model,padded_docs,labels,train_index,test_index)

Model: "model_17"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x_seq (InputLayer)              (None, 50)           0                                            
__________________________________________________________________________________________________
embedding_59 (Embedding)        (None, 50, 32)       64000       x_seq[0][0]                      
__________________________________________________________________________________________________
conv1d_36 (Conv1D)              (None, 49, 10)       650         embedding_59[0][0]               
__________________________________________________________________________________________________
conv1d_37 (Conv1D)              (None, 48, 10)       970         embedding_59[0][0]               
___________________________________________________________________________________________

In [187]:
from sklearn.model_selection import KFold

train_index = range(1050)
test_index = range(1050,len(padded_docs))
model = RNN(max_length,vocab_size,16)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',keras.metrics.Precision(),keras.metrics.Recall()])
model = model_fit(model,padded_docs,labels,train_index,test_index, 20)
# n_split=5
# for train_index,test_index in KFold(n_split).split(padded_docs):
#     # define the model
#     model = RNN(max_length,vocab_size,16)

#     model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',keras.metrics.Precision(),keras.metrics.Recall()])

#     model = model_fit(model,padded_docs,labels,train_index,test_index)

Model: "sequential_43"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_60 (Embedding)     (None, 50, 16)            32000     
_________________________________________________________________
gru_13 (GRU)                 (None, 16)                1584      
_________________________________________________________________
dense_83 (Dense)             (None, 1)                 17        
Total params: 33,601
Trainable params: 33,601
Non-trainable params: 0
_________________________________________________________________
None
Test set from  1050  to  1310
Epoch 1/20
1050/1050 [==============================] - 6s 6ms/step - loss: 0.6578 - accuracy: 0.6867 - precision_57: 0.2093 - recall_57: 0.0296
Epoch 2/20
1050/1050 [==============================] - 4s 4ms/step - loss: 0.6129 - accuracy: 0.7105 - precision_57: 0.0000e+00 - recall_57: 0.0000e+00
Epoch 3/20
1050/1050 [========================

In [188]:
from sklearn.model_selection import KFold


train_index = range(1050)
test_index = range(1050,len(padded_docs))
model_em = embedding(max_length,vocab_size,32)
model_em.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',keras.metrics.Precision(),keras.metrics.Recall()])
model_em = model_fit(model_em,padded_docs,labels,train_index,test_index)
# n_split=5
# for train_index,test_index in KFold(n_split).split(padded_docs):
#     # define the model
#     model = embedding(max_length,vocab_size,16)

#     model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy',keras.metrics.Precision(),keras.metrics.Recall()])
    
#     model = model_fit(model,padded_docs,labels,train_index,test_index)



Model: "sequential_44"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_61 (Embedding)     (None, 50, 32)            64000     
_________________________________________________________________
flatten_58 (Flatten)         (None, 1600)              0         
_________________________________________________________________
dense_84 (Dense)             (None, 1)                 1601      
Total params: 65,601
Trainable params: 65,601
Non-trainable params: 0
_________________________________________________________________
None
Test set from  1050  to  1310
Epoch 1/5
1050/1050 [==============================] - 2s 2ms/step - loss: 0.6453 - accuracy: 0.6886 - precision_58: 0.0741 - recall_58: 0.0066
Epoch 2/5
1050/1050 [==============================] - 0s 256us/step - loss: 0.5639 - accuracy: 0.7305 - precision_58: 1.0000 - recall_58: 0.0691 
Epoch 3/5
1050/1050 [==============================] 

In [189]:
import numpy
def predict_train(model, testData, truth):
    test = numpy.reshape(testData,(1,testData.shape[0]))
    pr = model.predict(test)
    print("==================")
    print("truth = ",truth )
    print("predict = ",pr )
    if (pr >= 0.5 and truth == 1) or (pr< 0.5 and truth ==0):
        print("true")
        return 1
    else:
        print("false")
        return 0


In [190]:
## CNN result
i = 0

for f, b in zip(padded_docs[1050:], labels[1050:]):
    i += predict_train(model_CNN,f,b)
print("total num:",len(padded_docs[1050:]))
print(i)

truth =  1
predict =  [[0.3993362]]
false
truth =  0
predict =  [[0.04696127]]
true
truth =  0
predict =  [[0.03834145]]
true
truth =  0
predict =  [[0.10807507]]
true
truth =  1
predict =  [[0.3833972]]
false
truth =  0
predict =  [[0.20861882]]
true
truth =  0
predict =  [[0.20066476]]
true
truth =  0
predict =  [[0.11861701]]
true
truth =  0
predict =  [[0.0840835]]
true
truth =  0
predict =  [[0.03399583]]
true
truth =  1
predict =  [[0.5667895]]
true
truth =  1
predict =  [[0.29123855]]
false
truth =  1
predict =  [[0.19440378]]
false
truth =  1
predict =  [[0.29480228]]
false
truth =  1
predict =  [[0.5323316]]
true
truth =  0
predict =  [[0.2834926]]
true
truth =  0
predict =  [[0.03370702]]
true
truth =  1
predict =  [[0.21261998]]
false
truth =  0
predict =  [[0.1042497]]
true
truth =  0
predict =  [[0.10484042]]
true
truth =  0
predict =  [[0.1553425]]
true
truth =  0
predict =  [[0.07002074]]
true
truth =  1
predict =  [[0.13961177]]
false
truth =  1
predict =  [[0.16372564]

truth =  1
predict =  [[0.2391827]]
false
truth =  0
predict =  [[0.17417167]]
true
truth =  0
predict =  [[0.1003234]]
true
truth =  0
predict =  [[0.07706827]]
true
truth =  1
predict =  [[0.37014452]]
false
truth =  0
predict =  [[0.1487223]]
true
truth =  0
predict =  [[0.12230556]]
true
truth =  0
predict =  [[0.26848644]]
true
truth =  0
predict =  [[0.06688989]]
true
truth =  0
predict =  [[0.08162066]]
true
truth =  0
predict =  [[0.07383226]]
true
truth =  1
predict =  [[0.272789]]
false
truth =  1
predict =  [[0.53217596]]
true
truth =  0
predict =  [[0.11639376]]
true
truth =  0
predict =  [[0.11230074]]
true
truth =  1
predict =  [[0.5270912]]
true
truth =  0
predict =  [[0.05507988]]
true
truth =  0
predict =  [[0.29956365]]
true
truth =  0
predict =  [[0.2582145]]
true
truth =  0
predict =  [[0.16324058]]
true
truth =  1
predict =  [[0.19465384]]
false
truth =  0
predict =  [[0.2793618]]
true
truth =  0
predict =  [[0.13424045]]
true
truth =  0
predict =  [[0.16711882]]
t

In [191]:
## emb result
i = 0
for f, b in zip(padded_docs[1050:], labels[1050:]):
    i += predict_train(model_em,f,b)
print("total num:",len(padded_docs[1050:]))
print(i)

truth =  1
predict =  [[0.32020545]]
false
truth =  0
predict =  [[0.0874698]]
true
truth =  0
predict =  [[0.10364649]]
true
truth =  0
predict =  [[0.11261543]]
true
truth =  1
predict =  [[0.6540091]]
true
truth =  0
predict =  [[0.24572307]]
true
truth =  0
predict =  [[0.08620015]]
true
truth =  0
predict =  [[0.09427688]]
true
truth =  0
predict =  [[0.07276823]]
true
truth =  0
predict =  [[0.10599101]]
true
truth =  1
predict =  [[0.5353088]]
true
truth =  1
predict =  [[0.63391286]]
true
truth =  1
predict =  [[0.5909491]]
true
truth =  1
predict =  [[0.55391204]]
true
truth =  1
predict =  [[0.41693422]]
false
truth =  0
predict =  [[0.2990215]]
true
truth =  0
predict =  [[0.09466054]]
true
truth =  1
predict =  [[0.35189784]]
false
truth =  0
predict =  [[0.07027189]]
true
truth =  0
predict =  [[0.38255996]]
true
truth =  0
predict =  [[0.08801344]]
true
truth =  0
predict =  [[0.1065345]]
true
truth =  1
predict =  [[0.15730934]]
false
truth =  1
predict =  [[0.15280932]]

truth =  1
predict =  [[0.5323896]]
true
truth =  0
predict =  [[0.08047029]]
true
truth =  0
predict =  [[0.625032]]
false
truth =  0
predict =  [[0.48416248]]
true
truth =  0
predict =  [[0.13374206]]
true
truth =  1
predict =  [[0.38065606]]
false
truth =  0
predict =  [[0.5345952]]
false
truth =  0
predict =  [[0.15587872]]
true
truth =  0
predict =  [[0.09795924]]
true
truth =  0
predict =  [[0.2994536]]
true
truth =  0
predict =  [[0.0918323]]
true
truth =  0
predict =  [[0.16313817]]
true
truth =  0
predict =  [[0.34513628]]
true
truth =  0
predict =  [[0.08797041]]
true
truth =  0
predict =  [[0.34165788]]
true
truth =  1
predict =  [[0.2036092]]
false
truth =  0
predict =  [[0.6668479]]
false
truth =  0
predict =  [[0.4451484]]
true
truth =  0
predict =  [[0.12166665]]
true
truth =  0
predict =  [[0.10547503]]
true
truth =  0
predict =  [[0.16800652]]
true
truth =  0
predict =  [[0.2408018]]
true
truth =  0
predict =  [[0.26274768]]
true
truth =  0
predict =  [[0.29232222]]
tr

In [192]:
## RNN result
i = 0
for f, b in zip(padded_docs[1050:], labels[1050:]):
    i += predict_train(model,f,b)
print("total num:",len(padded_docs[1050:]))
print(i)

truth =  1
predict =  [[0.61537814]]
true
truth =  0
predict =  [[0.00860798]]
true
truth =  0
predict =  [[0.01047687]]
true
truth =  0
predict =  [[0.00908913]]
true
truth =  1
predict =  [[0.84811544]]
true
truth =  0
predict =  [[0.7323004]]
false
truth =  0
predict =  [[0.00919424]]
true
truth =  0
predict =  [[0.01278028]]
true
truth =  0
predict =  [[0.0075246]]
true
truth =  0
predict =  [[0.0100442]]
true
truth =  1
predict =  [[0.84812057]]
true
truth =  1
predict =  [[0.84812057]]
true
truth =  1
predict =  [[0.84708446]]
true
truth =  1
predict =  [[0.83545804]]
true
truth =  1
predict =  [[0.84812033]]
true
truth =  0
predict =  [[0.686089]]
false
truth =  0
predict =  [[0.00992547]]
true
truth =  1
predict =  [[0.01154003]]
false
truth =  0
predict =  [[0.01103868]]
true
truth =  0
predict =  [[0.0152442]]
true
truth =  0
predict =  [[0.02374733]]
true
truth =  0
predict =  [[0.00882966]]
true
truth =  1
predict =  [[0.01496612]]
false
truth =  1
predict =  [[0.03931889]]

truth =  1
predict =  [[0.84812057]]
true
truth =  0
predict =  [[0.8481204]]
false
truth =  0
predict =  [[0.01632197]]
true
truth =  0
predict =  [[0.00990078]]
true
truth =  0
predict =  [[0.00759195]]
true
truth =  1
predict =  [[0.01594215]]
false
truth =  0
predict =  [[0.00939549]]
true
truth =  0
predict =  [[0.01959772]]
true
truth =  0
predict =  [[0.00895282]]
true
truth =  1
predict =  [[0.08090143]]
false
truth =  0
predict =  [[0.7913766]]
false
truth =  0
predict =  [[0.00927893]]
true
truth =  0
predict =  [[0.03284972]]
true
truth =  0
predict =  [[0.02033279]]
true
truth =  0
predict =  [[0.01056745]]
true
truth =  0
predict =  [[0.0067892]]
true
truth =  1
predict =  [[0.37201744]]
false
truth =  1
predict =  [[0.8481204]]
true
truth =  0
predict =  [[0.0375144]]
true
truth =  0
predict =  [[0.01204813]]
true
truth =  1
predict =  [[0.8481204]]
true
truth =  0
predict =  [[0.00819153]]
true
truth =  0
predict =  [[0.84811854]]
false
truth =  0
predict =  [[0.08779478

In [ ]:
# define documents
## read Data
SPL = "<SPL>"
path = "csv/trimedValues.csv"
train_file = path
dataframe = pd.read_csv(path, na_filter = False)
docs = []
labels = []
for i,data in dataframe.iterrows():
    d = data["comment"]
    c = data["code"]
    s = []
    s =  d+" "+SPL+" "+c # comment and code together
    docs.append(s)
    l = 1 if data["non-information"] == "yes" else 0
    labels.append(l)
# print(docs)
# define class labels
# print(label)
# integer encode the documents
vocab_size = 2000
encoded_docs = [one_hot(d, vocab_size) for d in docs]
dict_word = dict()
for d,e in zip(docs,encoded_docs):
    dict_word
    print(d)
    print(e)

# pad documents to a max length of 41 words
max_length = 41
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
# print(type(padded_docs))

In [208]:
# [1978, 206, 343, 193, 1548, 81]
s = "auto generated method stub <SPL> return"
one_hot(s, 2000)

[1978, 206, 343, 193, 1548, 81]